In [1]:
# imports
import numpy as np 


### Code for obtaining z component of gravity for the oblate ellipsoid. 

(the code needs) TO DO:

- Takes variables: a, b, c, x, y, z
    - a,b,c = ellipsoid parameters - for oblate ellispoid a < b = c
    - x, y, z = observation points, LOCAL coordinate system
        - make sure the nature of the x, y, z components is clear in docstring to avoid later confusion
- Computes the lambda parameter,
- Calculate z component gravity using GRAVMOD XIB equations from Clark et al. for oblate ellipsoids.
- Perform some sort of casual check
    - Eventually, will require a test.
- Will return a single component
    - A second definition to run this?
    - How is it best to store it?
    - Calculated within the definition? How complicated is too complicated for a function?

In [2]:
# variables

# dealing with outside the body - x,y,z must be on or outside of the defined ellipse
x = 10
y = 10
z = 10

# semi axes values are given when testing the functions as are different for each case.

In [3]:
def calculate_lambda(x, y, z, a, b, c): # takes semiaxes and observation coordinates
    
    """
    Calculate the value of lambda (parameter defining surfaces in confocal family,
    colloquially, the inflation or deflation parameter), for given ellipsoid semiaxes
    and given points of observation.
    x, y, z are positions in the local co-ordinate system.
    
    Parameters
    ----------
    Semiaxes (length) (integer): a, b, c
    Observation coordinates (integer): x, y, z


    Returns
    -------
    lambda (float): the value of lambda.
    
    """
    # Calculate lambda using x, y, z 
  
    p_0 = a**2 * b**2 * c**2 - b**2 * c**2 * x**2 - a**2 * b**2 * z**2
    p_1 = a**2 * b**2 + b**2 * c**2 + c**2 * a**2 - (b**2 + c**2) * x**2 - (c**2 + a**2) * y**2 - (a**2 + b**2) * z**2
    p_2 = a**2 + b**2 + c**2 - x**2 - y**2 - z**2

    p = p_1 - (p_2**2)/3 #check - is it just the p_3 term divided by 3?

    q = p_0 - ((p_1*p_2)/3) + 2*(p_2/3)**3

    theta = np.arccos(-q / (2 * np.sqrt((-p/3)**3))) #check - what is supposed to be included in the sqrt?
    
    lmbda = 2 * np.sqrt( (-p/3) * np.cos(theta/3) - p_2/3) #check - is the whole term included in the sqrt? Or just p_3?

    if not (lmbda > -c**2):
        raise ValueError(f"Lambda value is invalid: it should be true that lambda > -c**2, but instead lambda = {lmbda} and -c^2 = {-c**2}")
    

    return lmbda

In [4]:
# test - calculate lambda function

lambda_test = calculate_lambda(x, y, z, 10, 5, 5)
print(lambda_test)

# reasonable?

204.0803806285291


In [5]:
def calculate_delta_gs_oblate(x, y, z, a, b, c, density): # takes semiaxes, lambda value, density
    
    """
    Calculate the components of delta_g_i for i=1,2,3, for the oblate ellipsoid case (a < b = c 
    Delta_g_i represent the local axes system (for now, the only axes system). 
    x, y, z are positions in the local co-ordinate system.
    
    
    Parameters
    ----------
    lambda (float): the parameter defining surfaces in confocal family, for an ellipsoid.
    Observation coordinates (integer): x, y, z


    Returns
    -------
    # is this the best way to do this? will individual delta_g components be used later? maybe just return delta_g_z??
    
    dg1 (float): change in gravity for the x axis. (is this the right explanation?)
    dg2 (float): change in gravity for the y axis.
    dg3 (float): change in gravity for the z axis.

    """
    # constants
    G = 6.6743e-11

    # call and use lambda function 
    lmbda = calculate_lambda(x, y, z, a, b, c)
    
    # check the function is used for the correct type of ellipsoid
    if not (a < b and b == c):
        raise ValueError(f"Invalid ellipsoid axis lengths: expected a < b = c but got a = {a}, b = {b}, c = {c}")
    
    # compute the coefficient of the three delta_g equations 
    numerator = np.pi * a * b**2 * G * density
    denominator = (b**2 - a**2)**1.5 
    co_eff1 = numerator / denominator

    # compute repeated arctan term
    arc_tan_term = np.arctan(((b**2 - a**2) / (a**2 + lmbda))**0.5)
    
    # compute the terms within the brackets for delta_g 1,2,3
    bracket_term_g1 = arc_tan_term - ((b**2 - a**2) / (a**2 + lmbda) )**0.5
    
    bracket_term_g2g3 = ((((b**2 - a**2) * (a**2 + lmbda))**0.5) / (b**2 + lmbda)) - arc_tan_term

    # compile constants, coefficients, bracket terms to calculate final value of the delta_g terms
    dg1 = 4 * co_eff1 * x * bracket_term_g1
    dg2 = 2 * co_eff1 * y * bracket_term_g2g3
    dg3 = 2 * co_eff1 * z * bracket_term_g2g3

    return dg1, dg2, dg3

In [15]:
# test - calculate oblate delta_g function

dg1_ob, dg2_ob, dg3_ob = calculate_delta_gs_oblate(x, y, z, 2, 4, 4, density = 1000)
print(f'Vertical change in gravity for the oblate ellipsoid = {dg3_ob} mGal') 

# assuming this is producing a result in mGal, taking the upward as positive?

Vertical change in gravity for the oblate ellipsoid = -2.847021999376228e-08 mGal


In [10]:
def calculate_delta_gs_prolate(x, y, z, a, b, c, density): # takes semiaxes, lambda value, density
    
    """
    Calculate the components of delta_g_i for i=1,2,3, for the prolate ellipsoid case.
    Delta_g_i represent the local axes system (for now, the only axes system). 
    x, y, z are positions in the local co-ordinate system.
    
    Parameters
    ----------
    lambda (float): the parameter defining surfaces in confocal family, for an ellipsoid.
    Observation coordinates (integer): x, y, z


    Returns
    -------
    # is this the best way to do this? will individual delta_g components be used later? maybe just return delta_g_z??
    
    dg1 (float): change in gravity for the x axis. (is this the right explanation?)
    dg2 (float): change in gravity for the y axis.
    dg3 (float): change in gravity for the z axis.

    """
    # constants
    G = 6.6743e-11

    # call and use lambda function 
    lmbda = calculate_lambda(x, y, z, a, b, c)
    
    # check the function is used for the correct type of ellipsoid
    if not (a > b and b == c):
        raise ValueError(f"Invalid ellipsoid axis lengths: expected a > b = c but got a = {a}, b = {b}, c = {c}")
    
    # compute the coefficient of the three delta_g equations 
    numerator = np.pi * a * b**2 * G * density
    denominator = (a**2 - b**2)**1.5 
    co_eff1 = numerator / denominator

    # compute repeated log_e term
    log_term = np.log(((a**2 - b**2)**0.5 + (a**2 + lmbda)**0.5)/ ((b**2 + lmbda)**0.5))

    # compute repeated f_2 second term 
    f_2_term_2 = (((a**2 - b**2) * (a**2 + lmbda))**0.5)/(b**2 + lmbda)

    # compile terms 
    dg1 = 4 * co_eff1 * x * (((a**2 - b**2)/(a**2 + lmbda))**0.5 - log_term)
    dg2 = 2 * co_eff1 * y * (log_term - f_2_term_2) 
    dg3 = 2 * co_eff1 * z * (log_term - f_2_term_2) 
    
    return dg1, dg2, dg3

In [14]:
# test - calculate prolate delta_g function

dg1_pro, dg2_pro, dg3_pro = calculate_delta_gs_prolate(x, y, z, 4, 2, 2, density = 1000)
print(f'Vertical change in gravity for the prolate ellipsoid = {dg3_pro} mGal') 

# assuming this is producing a result in mGal, taking the upward as positive?

Vertical change in gravity for the prolate ellipsoid = -1.497883853004616e-08


In [ ]:
# not yet needed, but shell of final function to calculate the final component of gravity.

def calculate_oblate_gz(x, y, z, a, b, c): # takes semiaxes and observation coordinates
    
    """
    Compute the gravity field at a single point,
    given an oblate ellipsoid of uniform density.
    
    Parameters
    ----------
    Semiaxes (length) (integer): a, b, c
    Observation coordinates (integer): x, y, z


    Returns
    -------
    gz (float): vertical component of gravity, single value for a given 3D cooridnate system.
    lambda: ? If needed.
    
    """
    # call lambda function and get result
    lmbda = calculate_lambda(x, y, z, a, b, c)

    # call calculate gz components function and get results
    delta_z = dg1 * n1 + dg2 * n2 + dg3 * n3
    
    return gz

NOTES/QUESTIONS/THINGS TO CHECK:

- density units - g/cm^3 ??
- what does the n1 n2 n3 variables mean?
    - pretty sure its to do with the rotation matarix v...can ignore for now? just calculate as the magnitude?
- explanation/meaning of the delta_g1/2/3 terms?
- How is it best to structure these notebooks? Does it make sense to seperate by ellipsoid type?
